In [2]:
import json
import numpy as np
import math
import matplotlib.pyplot as plt
import os
from utils.visualization_utils import get_dict

%matplotlib inline
%load_ext autoreload
%autoreload 2

Using TensorFlow backend.


In [3]:
vrd_train_annotations = json.load(open("/data/ranjaykrishna/ReferringRelationships/data/VRD/annotations_train.json"))
vrd_test_annotations = json.load(open("/data/ranjaykrishna/ReferringRelationships/data/VRD/annotations_test.json"))
clevr_test_annotations = json.load(open("/data/ranjaykrishna/ReferringRelationships/data/clevr_color/annotations_test.json"))
#vg_train_annotations = json.load(open("/data/ranjaykrishna/ReferringRelationships/data/VisualGenome/VisualGenome/annotations_train.json"))
#vg_test_annotations = json.load(open("/data/ranjaykrishna/ReferringRelationships/data/VisualGenome/VisualGenome/annotations_test.json"))

In [4]:
vrd_obj_cat = json.load(open("/data/ranjaykrishna/ReferringRelationships/data/VRD/objects.json"))
clevr_obj_cat = json.load(open("/data/ranjaykrishna/ReferringRelationships/data/clevr_color/cat2obj.json"))

In [6]:
def update_dict(dic, k, v):
    if k in dic:
        if not v in dic[k]:
            dic[k] += [v]
    else:
        dic[k] = [v]
    return dic

def get_stats(annotations, data):
    images_list = []
    nb_im = 0
    nb_im_multiple_objects = 0
    for img in annotations:
        nb_im += 1
        objects_dict = {}
        for rel in annotations[img]:
            s_bbox = rel["subject"]["bbox"]
            o_bbox = rel["object"]["bbox"]
            s_cat = rel["subject"]["category"]
            o_cat = rel["object"]["category"]
            s_center = (int((s_bbox[0] + s_bbox[1])/2), int((s_bbox[2] + s_bbox[3])/2))
            o_center = (int((o_bbox[0] + o_bbox[1])/2), int((o_bbox[2] + o_bbox[3])/2))
            objects_dict = update_dict(objects_dict, s_cat, s_center)
            objects_dict = update_dict(objects_dict, o_cat, o_center)
        if len(objects_dict) < sum([len(x) for x in objects_dict.values()]):
            nb_im_multiple_objects += 1
            images_list += [img]
    print("Proportion of images with same object category appearing multiple times")
    print("{} : {}".format(data, 1.*nb_im_multiple_objects/nb_im))
    return images_list

def get_rel_multiple_objects(annotations, data):
    filtered_annotations = {}
    nb_rels = 0
    nb_rel_multiple_objects = 0
    for img in annotations:
        for rel in annotations[img]:
            nb_rels += 1
            s_bbox = rel["subject"]["bbox"]
            o_bbox = rel["object"]["bbox"]
            s_cat = rel["subject"]["category"]
            o_cat = rel["object"]["category"]
            s_center = (int((s_bbox[0] + s_bbox[1])/2), int((s_bbox[2] + s_bbox[3])/2))
            o_center = (int((o_bbox[0] + o_bbox[1])/2), int((o_bbox[2] + o_bbox[3])/2))
            if s_cat == o_cat:
                nb_rel_multiple_objects += 1
                if img in filtered_annotations:
                    filtered_annotations[img] += [rel]
                else:
                    filtered_annotations[img] = [rel]
    print("Proportion of relations with same object and subject category")
    print("{} : {}".format(data, 1.*nb_rel_multiple_objects/nb_rels))
    print("Total number of relationships: {}".format(nb_rels))
    return filtered_annotations

def get_annotation_obj_list(annotations, obj_list, data):
    filtered_annotations = {}
    nb_rels = 0
    nb_rel_in_cat = 0
    for img in annotations:
        for rel in annotations[img]:
            nb_rels += 1
            s_cat = vrd_obj_cat[rel["subject"]["category"]]
            o_cat = vrd_obj_cat[rel["object"]["category"]]
            if s_cat in obj_list or o_cat in obj_list:
                nb_rel_in_cat += 1
                if img in filtered_annotations:
                    filtered_annotations[img] += [rel]
                else:
                    filtered_annotations[img] = [rel]
    print("Proportion of relations with object or subject category in:\n {}".format(obj_list))
    print("{} : {}".format(data, 1.*nb_rel_in_cat/nb_rels))
    print("Total number of relationships: {}".format(nb_rels))
    print("Number of relationships kept: {}".format(nb_rel_in_cat))
    return filtered_annotations

In [46]:
def get_rel_subset(annotations, data):
    filtered_annotations = {}
    nb_rel = 0
    nb_im_multiple_objects = 0
    for img in list(annotations.keys()):
        objects_list = {}
        for rel in annotations[img]:
            nb_rel += 1
            s_bbox = rel["subject"]["bbox"]
            o_bbox = rel["object"]["bbox"]
            s_cat = rel["subject"]["category"]
            o_cat = rel["object"]["category"]
            s_center = (int((s_bbox[0] + s_bbox[1])/2), int((s_bbox[2] + s_bbox[3])/2))
            o_center = (int((o_bbox[0] + o_bbox[1])/2), int((o_bbox[2] + o_bbox[3])/2))
            objects_list = update_dict(objects_list, s_cat, s_center)
            objects_list = update_dict(objects_list, o_cat, o_center)
        multiple_objects = []
        for x in objects_list:
            if len(objects_list[x]) > 1:
                for center in objects_list[x]:
                    multiple_objects += [(x, center)]
        for rel in annotations[img]:
            s_bbox = rel["subject"]["bbox"]
            o_bbox = rel["object"]["bbox"]
            s_cat = rel["subject"]["category"]
            o_cat = rel["object"]["category"]
            s_center = (int((s_bbox[0] + s_bbox[1])/2), int((s_bbox[2] + s_bbox[3])/2))
            o_center = (int((o_bbox[0] + o_bbox[1])/2), int((o_bbox[2] + o_bbox[3])/2))
            if (s_cat, s_center) in multiple_objects or (o_cat, o_center) in multiple_objects:
                nb_im_multiple_objects += 1
                filtered_annotations = update_dict(filtered_annotations, img, rel)
    print("Proportion of relationships with object category appearing multiple times")
    print("{} : {}".format(data, 1.*nb_im_multiple_objects/nb_rel))
    return filtered_annotations

In [47]:
filtered_annotations = get_rel_subset(clevr_test_annotations, "clevr")

Proportion of relationships with object category appearing multiple times
clevr : 0.8177356596776401


In [49]:
json.dump(filtered_annotations, open('/data/chami/ReferringRelationships/data/Clevr/test_annotation_multi.json', 'w'))

In [50]:
filtered_annotations = get_rel_subset(vrd_test_annotations, "vrd")

Proportion of relationships with object category appearing multiple times
vrd : 0.5806493846556691


In [53]:
json.dump(filtered_annotations, open('/data/chami/ReferringRelationships/data/VRD/annotations_test_multi.json', 'w'))

In [52]:
len(vrd_test_annotations)

1000

In [45]:
len(clevr_test_annotations)

15000

In [41]:
filtered_annotations['10008170616_a54deca9e3_b.jpg']

[{'object': {'bbox': [176, 265, 4, 102], 'category': 10},
  'predicate': 1,
  'subject': {'bbox': [101, 601, 1, 110], 'category': 0}},
 {'object': {'bbox': [293, 591, 2, 68], 'category': 6},
  'predicate': 1,
  'subject': {'bbox': [101, 601, 1, 110], 'category': 0}},
 {'object': {'bbox': [389, 435, 721, 747], 'category': 52},
  'predicate': 27,
  'subject': {'bbox': [241, 550, 707, 846], 'category': 0}},
 {'object': {'bbox': [320, 361, 953, 1021], 'category': 62},
  'predicate': 2,
  'subject': {'bbox': [267, 675, 865, 1024], 'category': 0}},
 {'object': {'bbox': [267, 675, 865, 1024], 'category': 0},
  'predicate': 0,
  'subject': {'bbox': [320, 361, 953, 1021], 'category': 62}},
 {'object': {'bbox': [267, 675, 865, 1024], 'category': 0},
  'predicate': 0,
  'subject': {'bbox': [492, 554, 960, 996], 'category': 95}}]

In [14]:
obj_list = ["hydant", "faucet", "luggage", "suitcase", "ramp", "grass", "sky"]
filtered_annotations = get_annotation_obj_list(vrd_test_annotations, obj_list, "vrd")

Proportion of relations with object or subject category in:
 ['hydant', 'faucet', 'luggage', 'suitcase', 'ramp', 'grass', 'sky']
vrd : 0.11861743912018853
Total number of relationships: 7638
Number of relationships kept: 906


In [25]:
json.dump(ann, open('/data/chami/ReferringRelationships/data/vrd_rels_multiple_objects.json', 'w'))

In [8]:
test_vrd_images = get_stats(vrd_test_annotations, "vrd")

Proportion of images with same object category appearing multiple times
vrd : 0.612


In [106]:
len(test_vg_images)

5160

In [102]:
json.dump(test_vrd_images, open('/data/chami/ReferringRelationships/data/vrd_images_multiple_objects.json', 'w'))

In [109]:
json.dump(test_vg_images, open('/data/chami/ReferringRelationships/data/visual_genome_images_multiple_objects.json', 'w'))